In [ ]:
import pandas as pd
import numpy as np
import jieba
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
from keras import layers
from keras import regularizers
from sklearn import metrics

In [ ]:
df=pd.read_csv('train.csv',encoding = 'utf-8')
df=df.sample(frac=1)

In [ ]:
x_train=[]
y_train=[]
x=[]

In [ ]:
for i in range(len(df)):
     x.append(df['label\tcomment'][i].split('\t')[1])
     x_train.append(' '.join(jieba.cut(x[i],cut_all=False)))
     y_train.append(df['label\tcomment'][i].split('\t')[0])

In [ ]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [ ]:
stop_words_file = "哈工大停用词表.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [ ]:
fil_x=[]
for i in range(len(x_train)):
    q=x_train[i].split()
    strr=''
    for word in q:
        if word not in stopwords:
            strr=strr+word+' '
    fil_x.append(strr)

In [ ]:
data_y=np.asarray(y_train)

In [ ]:
num_words =12000   #设置的最大词数
max_len=80

tk = Tokenizer(num_words=num_words+1)  #因为要加未登录词，所以+1

tk.fit_on_texts(fil_x)
tk.word_index[tk.oov_token] = num_words + 1 
sequence=tk.texts_to_sequences(fil_x)
data_x=pad_sequences(sequence,maxlen=max_len)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, random_state=1,train_size=0.9,test_size=0.1)

In [ ]:
dff=pd.read_csv('test_new.csv',encoding = 'utf-8')
id=[]
x_test=[]
x1=[]
for i in range(len(dff)):
     x1.append(dff['comment'][i])
     id.append(dff['id'][i])
     x_test.append(' '.join(jieba.cut(x1[i],cut_all=False)))

In [ ]:
fil_x_test=[]
for i in range(len(x_test)):
    q=x_test[i].split()
    strr=''
    for word in q:
        if word not in stopwords:
            strr=strr+word+' '
    fil_x_test.append(strr)

In [ ]:
sequence1=tk.texts_to_sequences(fil_x_test)
data_x_test=pad_sequences(sequence1,maxlen=max_len)

In [ ]:
from gensim.models import KeyedVectors
zh_model = KeyedVectors.load_word2vec_format('zh.vec')

In [ ]:
import numpy as np
print(len(zh_model[next(iter(zh_model.vocab))]))
embedding_dim = len(zh_model[next(iter(zh_model.vocab))])
embedding_matrix = np.random.rand(num_words, embedding_dim)
embedding_matrix = (embedding_matrix - 0.5) * 2

In [1]:
model=Sequential()
model.add(Embedding(num_words,embedding_dim)
model.add(layers.Bidirectional(layers.LSTM(50,dropout=0.1,recurrent_dropout=0.5)))#output_dim=200
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

history=model.fit(X_train,Y_train,epochs=20,batch_size=520,validation_data=(X_test,Y_test))

SyntaxError: invalid syntax (<ipython-input-1-1ade85f8c37d>, line 3)

In [ ]:
result1=model.evaluate(X_test,Y_test)
print(result1)

In [ ]:

units = 32

model = Sequential()
model.add(Embedding(num_words, embedding_dim))
model.add(LSTM(units))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(X_train, Y_train,
                    epochs=20,
                    batch_size=1024,
                    validation_data=(X_test, Y_test))
result10=model.evaluate(X_test,Y_test)



In [ ]:
print (result10)

In [ ]:
from keras.optimizers import RMSprop
model=Sequential()
model.add(Embedding(num_words,120,input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(50,dropout=0.1,recurrent_dropout=0.5)))#output_dim=60
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

history=model.fit(X_train,Y_train,epochs=20,batch_size=1024,validation_data=(X_test,Y_test))
result1=model.evaluate(X_test,Y_test)


In [ ]:
result1

In [ ]:
import matplotlib.pyplot as plt
history_dict=history.history
loss_values=history_dict['loss']
val_loss_values=history_dict['val_loss']
epochs=range(1,len(loss_values)+1)
plt.plot(epochs,loss_values,'bo',label='Training Loss')
plt.plot(epochs,val_loss_values,'b',label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc=history_dict['acc']
val_acc=history_dict['val_acc']
plt.plot(epochs,acc,'bo',label="Training acc")
plt.plot(epochs,val_acc,label='Validation acc')
plt.title("Traing and Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model=Sequential()
model.add(Embedding(num_words,120,input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(50,dropout=0.1,recurrent_dropout=0.5)))#output_dim=60
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(X_train,Y_train,epochs=15,batch_size=1024,validation_data=(X_test,Y_test))

In [ ]:
result=model.evaluate(X_test,Y_test)
print(result)

In [ ]:
features1=model.predict(data_x_test)
output1=[]
for i in range(len(features1)):
    if features1[i][0]>0.5 or features1[i][0]==0.5:
        output1.append(1)
    else :
        output1.append(0)
        

In [ ]:
model=Sequential()
model.add(Embedding(num_words,120,input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(50,dropout=0.1,recurrent_dropout=0.5)))#output_dim=60
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(X_train,Y_train,epochs=15,batch_size=1024,validation_data=(X_test,Y_test))

In [ ]:
result=model.evaluate(X_test,Y_test)
print(result)

In [ ]:
features5=model.predict(data_x_test)
output5=[]
for i in range(len(features5)):
    if features5[i][0]>0.5 or features5[i][0]==0.5:
        output5.append(1)
    else :
        output5.append(0)
        

In [ ]:
import pandas as pd
df3=pd.read_csv("correct.csv",encoding='utf-8',sep=',')
df3=df3['label']
df3array=df3.values
df3array

In [ ]:
TN=0
TP=0
FP=0
FN=0

for i in range(len(output1)):
    if (output1[i]==0)and(int(df3array[i])==0):
        TN=TN+1
    if (output1[i]==1)and(int(df3array[i])==1):
        TP=TP+1
    if (output1[i]==0)and(int(df3array[i])==1):
        FN=FN+1
    if (output1[i]==1)and(int(df3array[i])==0):
        FP=FP+1;
acc=(TP/(TP+FP))
callbackrate=TP/(TP+FN)
score=(2*acc*callbackrate)/(acc+callbackrate)
print(score)

In [ ]:
(output1)

In [ ]:
print(len(id))
print(len(output1))

In [ ]:
dataFrameToSave=pd.DataFrame({'id':id,'label':output1})
dataFrameToSave.shape
dataFrameToSave.to_csv("sample.csv",index=False,sep=",")

In [ ]:
dataFrameToSave